In [ ]:

from zipfile import ZipFile
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms import Resize
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.optim as optim
import numpy as np
from torch.utils.data import Subset
import matplotlib.pyplot as plt
import os
import cv2
import shutil
from torch.utils.data import random_split

%pip install transformers

from transformers import ViTModel, ViTFeatureExtractor
import torch.nn as nn

# Load a pretrained ViT model
vit = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

#set use_cuda to true for GPU use
use_cuda = True

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
%pip install split-folders
import splitfolders

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#This ended up not being used
'''def folder_combiner(folder_path, destination_path):
  for emotion_folder in os.listdir(folder_path):
    emotion_folder_path = os.path.join(folder_path, emotion_folder)
    if os.path.isdir(emotion_folder_path):
        # Iterate through images in each emotion folder
        for filename in os.listdir(emotion_folder_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):  # Add more extensions if needed
                src_path = os.path.join(emotion_folder_path, filename)
                dst_path = os.path.join(destination_path, f"{emotion_folder}_{filename}")  # Prefix with emotion label
                shutil.copy(src_path, dst_path)'''

In [ ]:
train_folder_path = '/content/gdrive/My Drive/APS360 Project/kaggle_facial_emotion_data/train'
test_folder_path = '/content/gdrive/My Drive/APS360 Project/test'
test_destination_path = '/content/gdrive/My Drive/APS360 Project/Combined Data'
train_destination_path = ''

#folder_combiner(test_folder_path, test_destination_path)

In [ ]:
#Function to combine all of the class folders into a single folder
#Was supposed to do same thing as above function, also not used.
'''def load_images(folder_path):
    images = []
    labels = []
    emotion_folders = os.listdir(folder_path)
    for emotion_folder in emotion_folders:
        emotion_path = os.path.join(folder_path, emotion_folder)
        if os.path.isdir(emotion_path):
            label = emotion_folder  # Use folder name as the label
            for filename in os.listdir(emotion_path):
                img_path = os.path.join(emotion_path, filename)
                if img_path.endswith(".jpg") or img_path.endswith(".png"):  # Add more extensions if needed
                    img = cv2.imread(img_path)
                    if img is not None:
                        images.append(img)
                        labels.append(label)
    return images, labels'''

In [ ]:
splitfolders.ratio('/content/gdrive/MyDrive/APS360 Project/Data/train', output="split_data",
    seed=999, ratio=(0.8, 0.1, 0.1), group_prefix=None, move=False)

In [ ]:
transform = transforms.Compose(
        [transforms.ToTensor(),transforms.Resize((224,224))])

#transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224,224))])

#Create train, validation, and testing datasets
#Apply the transformations

train_data = ImageFolder("/content/gdrive/MyDrive/APS360 Project/Data/train", transform)

train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size

train_data, val_data = random_split(train_data, [train_size, val_size])
test_data = ImageFolder("/content/gdrive/MyDrive/APS360 Project/Data/test", transform)

#Load all of the datasets into their respective loaders
#train_loader = torch.utils.data.DataLoader(train_data, batch_size=10,
                                           #shuffle=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=10, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=10,
                                         shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=10,
                                          shuffle=True)

In [ ]:
class HybridEmotionClassifier(nn.Module):
    def __init__(self, name="HybridEmotionClassifier"):
        super(HybridEmotionClassifier, self).__init__()
         # CNN components
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        # Fully connected layers
        self.fc1_cnn = nn.Linear(10 * 54 * 54, 128)  # 29160 -> 128
        self.fc2_cnn = nn.Linear(128, 64)            # 128 -> 64

        # ViT components
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        self.fc1_vit = nn.Linear(768, 32)  # 768 is the output dimension of ViT

        # Combined fully connected layer
        self.fc_combined = nn.Linear(96, 7)  # Adjusted from 64 to 96

    def forward(self, x):
        # CNN forward pass
        x_cnn = self.pool(F.relu(self.conv1(x)))
        x_cnn = self.pool(F.relu(self.conv2(x_cnn)))
        x_cnn = x_cnn.view(x_cnn.size(0), -1)  # Flatten the output for the fully connected layer
        x_cnn = F.relu(self.fc1_cnn(x_cnn))
        x_cnn = F.relu(self.fc2_cnn(x_cnn))

        # ViT forward pass
        vit_outputs = self.vit(pixel_values=x)['last_hidden_state']
        x_vit = vit_outputs[:, 0]  # [CLS] token output
        x_vit = F.relu(self.fc1_vit(x_vit))

        # Concatenate CNN and ViT features
        x_combined = torch.cat((x_cnn, x_vit), dim=1)

        # Final classification layer
        x_out = self.fc_combined(x_combined)

        return x_out

In [ ]:
#Get model name for checkpointing
def get_model_name(name, batch_size, learning_rate, epoch):
   path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size,
                                                  learning_rate, epoch)
   return path

In [ ]:
#Find the accuracy of the model
def get_accuracy(model, data_loader):
    #initialize counters
    correct = 0
    total = 0
    for imgs, labels in data_loader:
        #Enable GPU if available
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()

        output = model(imgs)
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    #Return accuracy
    return correct / total

In [ ]:
#plot accuracy
def plot_training_curve(path, num_epochs, train_acc, val_acc):
    plt.title("Training Curve")
    n = len(num_epochs) # number of epochs
    plt.plot(range(1,n+1), train_acc, label="Train")
    plt.plot(range(1,n+1), val_acc, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

In [ ]:
def train(model, train_loader, val_loader, batch_size=64, num_epochs=15,
          learn_rate=0.001, model_name='HybridEmotionClassifier'):
    #Set seed
    torch.manual_seed(1700)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learn_rate)

    #populate accuracy arrays with zeroes
    train_acc = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)

    #initialize iteration counter
    n = 0
    for epoch in range(num_epochs):
        for images, labels in iter(train_loader):
            #Enable GPU if available
            if use_cuda and torch.cuda.is_available():
              images = images.cuda()
              labels = labels.cuda()

            #perform forwards pass
            out = model(images)
            #Compute loss using Cross Entropy loss function
            loss = criterion(out, labels)
            #backward pass to calculate and update parameters.
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            #increment iteration counter
            n += 1

        # track training and validation accuracy
        train_acc[epoch] = get_accuracy(model, train_loader)
        val_acc[epoch] = get_accuracy(model, val_loader)

        #output accuracy
        print(("Epoch: {} | Training acc: {} |" + "Validation acc: {}"
        ).format(epoch, train_acc[epoch], val_acc[epoch]))

        #checkpoint model
        model_path = get_model_name(model_name, batch_size, learn_rate,
                                    epoch)
        torch.save(model.state_dict(), model_path)

    epochs = np.arange(1, num_epochs + 1)

    #plot training curve
    plot_training_curve(model_path, epochs, train_acc, val_acc)

In [ ]:
#Overfitting on small dataset for sanity check

model = HybridEmotionClassifier()

#sample 10 random indices from the training data
indices = np.random.choice(len(train_data), size=10, replace=False)
#create an overfit dataset as a subset of the training dataset
overfit_data = Subset(train_data, indices)
#Load the overfit dataset
overfit_loader = DataLoader(overfit_data, batch_size=2, shuffle=True)

#Enable GPU if available.
if use_cuda and torch.cuda.is_available():
  model.cuda()

train(model, overfit_loader, overfit_loader, num_epochs=30, batch_size=27,
      learn_rate=0.001)
#Please note that since I passed the overfit_loader for both the
#parameters train_loader and val_loader, the training and validation
#accuracy seen here will be identical because they are the same thing

In [ ]:
model1 = HybridEmotionClassifier()
if use_cuda and torch.cuda.is_available():
  model1.cuda()

train(model1, train_loader, val_loader, num_epochs = 30)

In [ ]:
path = get_model_name(model1.name, batch_size=64, learning_rate=0.001,
                      epoch=11)
state = torch.load(path)
model1.load_state_dict(state)

criterion = nn.CrossEntropyLoss()
test_acc = get_accuracy(model1, test_loader)
print(test_acc)